<p style="text-align: center;font-size: 40pt">TF<\p>
<p style="text-align: center;font-size: 20pt">Library for keeping track of multiple coordinate frames over time<\p>
<img src="images/tf_title.png" width="100%">

Robots usually consist of different parts which move relative to each other.
Mobile robots move around and change their position relative to some given world coordinate frame.
To track all this motion, it is natural to attach coordinate frames to the moving parts (or whole robots) and to describe their relative poses by rigid transformations.
Thanks to the modules [Welcome to a flat world](../1-transformations_2d/1.0-overview.ipynb) of this course, you are getting a good overview of how rigid transformations work.
In ROS, the task of managing all the information related to coordinate frames and transformations between them is handled by the TF library.

First of all, visit the [wiki.ros.org/tf2](http://wiki.ros.org/tf2) page and see how the authors of TF introduce it.
We are providing a link to _TF2_ which is a successor of the TF in several latest versions of ROS.
The changes in TF2 are mainly technical, so this is just to inform you that two versions exist.

From the user perspective, the mechanisms of TF are usually hidden and used internally by running ROS nodes.
It is however good to have a high-level understanding of the system to be able to detect and fix issues.

# How it works internally

By design, TF is a decentralized system, where each running ROS node that uses TF keeps its own buffer of available transformations.
By default, this buffer keeps 20 seconds of history.
The transformations are published on the `/tf` and `/tf_static` topics, and all TF-using nodes listen to these topics.
Each transform contains a time stamp, name of the coordinate frames (parent and child) and the actual transformation parameters (i.e., a translation vector and a quaternion encoding the rotation).
Each coordinate frame is allowed to have maximum one parent frame.
This constraint is a design decision taken by the authors of TF to avoid loop in the connection tree.
Therefore, the whole set of coordinate frames is organized as directed tree, with no cycles possible.
This helps with effective implementation and does not seriously limit the usability of the system.

As en example, here is a transformation published on the `/tf` topic:
```
---
transforms: 
  - 
    header: 
      seq: 0
      stamp: 
        secs: 1582746617
        nsecs: 256056070
      frame_id: "odom"
    child_frame_id: "base_link"
    transform: 
      translation: 
        x: -2.67658308705
        y: 41.3771464098
        z: 0.974358148222
      rotation: 
        x: -0.00541755560712
        y: 0.00241126321715
        z: 0.910911088387
        w: 0.412560086478
---
```

You can see for yourself by executing `rostopic echo /tf` while having the rosbag from the Rviz lesson running.
The particular transformation shown here comes from a node that handles odometry (i.e., an estimate of a pose of a mobile robot in a fixed world frame).
It says what is the expected position and orientation of the robot, named `base_link`, in the fixed coordinate frame, named `odom`, at that given time.

The TF then offers functions to find transformations between any two coordinate frames for any time present in the local buffer (from now to those 20 default seconds in history).
The size of the buffer can be increased, but from our experience, usually there is no need to.
The library also offers functions to transform vector or point coordinates between different frames.
This is however all done in code, which is out of the scope of this lesson.
You might still want to be able to visualize the coordinate frames...

# How to see what is going on in TF

There are four ways to investigate what is going on with TF.
We will explain them one by one.

## Using Rviz

The most preferred way is to see the frames live in Rviz as they express 3D quantities.
Our lesson on Rviz shows you how to do it and you should be able to see something similar to what the title image in this notebook shows.
What moves and what stays fixed in the origin of the Rviz render depends on your setting of the _Fixed Frame_ in Rviz.
That way, you can make the robot coordinate frame move inside the `odom` or `map` frame, or you can make the `map` and `odom` frames move around the robot, which would be fixed in the origin.
Keep in mind that this is only a point of view, as frame of reference is a relative concept.
The underlying system of transformations stays the same.

## Using `rosrun tf view_frames`

The ROS package TF contains a neat tool to visualize the TF tree.
This comes handy when you cannot use Rviz or when you need a snapshot of the TF system.
By executing `rosrun tf view_frames`, you will obtain a pdf file with all the frames the TF heard about in a five-second-long period after executing the command.
The pdf is saved in the current folder you executed the command from.
For example, running the command on our small example bag file gives us this result:

<img src="images/tf_tree.png" width="100%" style="display:block; margin: 0 auto;">

To try it for yourself, you will need to first run a rosbag and then do
```
$ rosrun tf view_frames
<wait 5 sec>
$ evince frames.pdf
```

The last command use `evince`, which is simply the pdf viewer in Ubuntu.
As you can see, we have a lot of coordinate frames and thus the pdf needs some zooming.

## Using `rosrun rqt_tf_tree rqt_tf_tree`

This way of displaying the TF tree is very similar to the previous one, but uses a dedicated GUI instead of a pdf.
You can run the visualization by executing `rosrun rqt_tf_tree rqt_tf_tree`.
When using our example rosbag file, the output looks like this:

<img src="images/rqt_tf_tree.png" width="100%" style="display:block; margin: 0 auto;">

As you can see, the output looks almost identical to the `rosrun tf view_frames`, but it has additional features, such as highlighting related frames when moving the mouse over them and also exporting the tree to different file formats. 

## Looking up a transformation in a terminal

Finally, if you need to know the transformation values exactly, you can _echo_ the transformation to the terminal by executing `rosrun tf tf_echo [source_frame] [target_frame]`.
Again using our example rosbag file and calling `rosrun tf tf_echo /odom /base_link` while the bag is playing gives you this output:

```
At time 1582746646.255
- Translation: [-8.619, 46.759, 1.132]
- Rotation: in Quaternion [-0.004, 0.004, 0.962, 0.272]
            in RPY (radian) [0.007, 0.010, 2.590]
            in RPY (degree) [0.378, 0.554, 148.397]
At time 1582746647.000
- Translation: [-8.619, 46.759, 1.132]
- Rotation: in Quaternion [-0.004, 0.005, 0.962, 0.272]
            in RPY (radian) [0.007, 0.010, 2.590]
            in RPY (degree) [0.397, 0.590, 148.420]
At time 1582746648.003
- Translation: [-8.619, 46.759, 1.132]
- Rotation: in Quaternion [-0.004, 0.005, 0.962, 0.272]
            in RPY (radian) [0.007, 0.010, 2.591]
            in RPY (degree) [0.407, 0.586, 148.475]
At time 1582746648.991
- Translation: [-8.619, 46.759, 1.132]
- Rotation: in Quaternion [-0.004, 0.005, 0.962, 0.272]
            in RPY (radian) [0.007, 0.010, 2.592]
            in RPY (degree) [0.417, 0.596, 148.490]
````

from which you can read the translation and rotation between the requested coordinate frames.

# Wrap-up

This lesson has shown you how to visualize available transformations in TF.

To train a little more with TF, go to the [exercise](../../exercises/ex_ros/4e-exercises_tf.ipynb) where we will try a ROS tutorial on TF ;)

Then, the last lesson will be:
- [Processing data (roslaunch)](2.5-lesson-ros-roslaunch.ipynb)

&#8635; [Go back to the list of lessons](2.0-overview.ipynb)